In [ ]:
!pip install emoji
!pip install tensorflow
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

2022-04-20 00:07:04.764819: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-20 00:07:04.765021: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("Train_Dec.csv")

In [ ]:
df['text'] = df['text'].astype('str') 

In [ ]:
docs = list(df.loc[:, "text"].values)

In [ ]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|███████████████████████████████████| 56131/56131 [2:09:10<00:00,  7.24it/s]


In [ ]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)

In [ ]:
df

,text,Label,Negative,Neutral,Positive
0,@Alyssafarah They started the armed conflict a...,0,0.926742,0.070582,0.002675
1,Tucker Carlson emphasized on how much Ukraine ...,1,0.350306,0.642104,0.007590
2,"Since day one, the military fighting against U...",0,0.759732,0.236510,0.003758
3,A notable Russian OSS developer promotes a pro...,0,0.499068,0.492452,0.008480
4,US believes Russia could invade Ukraine ‘as so...,1,0.190029,0.803336,0.006634
...,...,...,...,...,...
56126,@20committee Russia could roll through Ukraine...,1,0.447488,0.545092,0.007420
56127,Russia Preparing 175K Troops For Ukraine Offen...,1,0.095686,0.896965,0.007348
56128,"@politicalblond In other words, Russia is Male...",0,0.946390,0.050894,0.002716
56129,How the US Military Could Mobilize If Russia I...,1,0.040387,0.950251,0.009362


In [ ]:
df.to_csv('finetuned_Bertweet_SA_Dec_Train.csv')